In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pylab as plt

from keras.utils import np_utils 

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Data

In [2]:
dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"
source_path = "I:\\Honours-Project\\data\\sorted\\agg\\cropped\\"
csv = "I:\\Honours-Project\\data\\sorted\\agg\\ids.csv"

In [3]:
df = pd.read_csv(csv,index_col=0).sample(frac=1)

In [4]:
df = df[df['name'].str.contains("_flipped")]

In [11]:
ds_count = min(df.groupby('date').size())
ds_flow = df.groupby('date').size().idxmin()

samples = pd.concat([df[df['date']==i].sample(n=ds_count) for i in df.date.unique()],ignore_index=True)

df = samples.sample(frac=1)

print(df.groupby('date').size())

date
0    22
1    22
2    22
3    22
dtype: int64


In [6]:
nb_classes = 4

x = []
y = []

df['log_volume'] = df['volume'].apply(np.log)

mat = df.drop(columns=['name','date','flow_rate','source','moments','inertia_tensor'])

# Drop catagorical features
mat = mat.drop(columns=['cluster'])

X = mat.as_matrix()
# x = standardize = preprocessing.scale(x)

y = df["date"].values

y[y == '2017.11.09'] = 0
y[y == '2017.11.17'] = 1
y[y == '2017.11.22'] = 2
y[y == '2017.11.23'] = 3

one_hot = np_utils.to_categorical(df["date"].values, nb_classes)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=36, stratify=y)

In [8]:
stats.itemfreq(y_test)

array([[0, 9],
       [1, 9],
       [2, 9],
       [3, 9]], dtype=object)

In [9]:
X_train.shape

(52, 13)

In [10]:
tpot = TPOTClassifier(generations=100, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_mnist_pipeline.py')

RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly.